<a href="https://colab.research.google.com/github/dajulie/PyTorchZeroToAll/blob/main/lec_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from torch import nn, tensor, max
import numpy as np

Y = np.array([1, 0, 0])
Y_pred1 = np.array([0.7, 0.2, 0.1])
Y_pred2 = np.array([0.1, 0.3, 0.6])
print(f'Loss1: {np.sum(-Y * np.log(Y_pred1)):.4f}')
print(f'Loss2: {np.sum(-Y * np.log(Y_pred2)):.4f}')



loss = nn.CrossEntropyLoss()

Y = tensor([0], requires_grad=False)

Y_pred1 = tensor([[2.0, 1.0, 0.1]])
Y_pred2 = tensor([[0.5, 2.0, 0.3]])

l1 = loss(Y_pred1, Y)
l2 = loss(Y_pred2, Y)

print(f'PyTorch Loss1: {l1.item():.4f} \nPyTorch Loss2: {l2.item():.4f}')
print(f'Y_pred1: {max(Y_pred1.data, 1)[1].item()}')
print(f'Y_pred2: {max(Y_pred2.data, 1)[1].item()}')



Y = tensor([2, 0, 1], requires_grad=False)

Y_pred1 = tensor([[0.1, 0.2, 0.9],
                  [1.1, 0.1, 0.2],
                  [0.2, 2.1, 0.1]])

Y_pred2 = tensor([[0.8, 0.2, 0.3],
                  [0.2, 0.3, 0.5],
                  [0.2, 0.2, 0.5]])

l1 = loss(Y_pred1, Y)
l2 = loss(Y_pred2, Y)
print(f'Batch Loss1: {l1.item():.4f} \nBatch Loss2: {l2.data:.4f}')


Loss1: 0.3567
Loss2: 2.3026
PyTorch Loss1: 0.4170 
PyTorch Loss2: 1.8406
Y_pred1: 0
Y_pred2: 1
Batch Loss1: 0.4966 
Batch Loss2: 1.2389


In [4]:
from __future__ import print_function
from torch import nn, optim, cuda
from torch.utils import data
from torchvision import datasets, transforms
import torch.nn.functional as F
import time

batch_size = 64
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST Model on {device}\n{"=" * 44}')

train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

train_loader = data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 520)
        self.l2 = nn.Linear(520, 320)
        self.l3 = nn.Linear(320, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)


model = Net()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += criterion(output, target).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')


if __name__ == '__main__':
    since = time.time()
    for epoch in range(1, 5):
        epoch_start = time.time()
        train(epoch)
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Training time: {m:.0f}m {s:.0f}s')
        test()
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Testing time: {m:.0f}m {s:.0f}s')

    m, s = divmod(time.time() - since, 60)
    print(f'Total Time: {m:.0f}m {s:.0f}s\nModel was trained on {device}!')

Training MNIST Model on cpu
Train Epoch: 1 | Batch Status: 0/60000 (0%) | Loss: 2.303748
Train Epoch: 1 | Batch Status: 640/60000 (1%) | Loss: 2.299030
Train Epoch: 1 | Batch Status: 1280/60000 (2%) | Loss: 2.298820
Train Epoch: 1 | Batch Status: 1920/60000 (3%) | Loss: 2.308497
Train Epoch: 1 | Batch Status: 2560/60000 (4%) | Loss: 2.305350
Train Epoch: 1 | Batch Status: 3200/60000 (5%) | Loss: 2.287960
Train Epoch: 1 | Batch Status: 3840/60000 (6%) | Loss: 2.305628
Train Epoch: 1 | Batch Status: 4480/60000 (7%) | Loss: 2.302193
Train Epoch: 1 | Batch Status: 5120/60000 (9%) | Loss: 2.294440
Train Epoch: 1 | Batch Status: 5760/60000 (10%) | Loss: 2.296069
Train Epoch: 1 | Batch Status: 6400/60000 (11%) | Loss: 2.304749
Train Epoch: 1 | Batch Status: 7040/60000 (12%) | Loss: 2.294084
Train Epoch: 1 | Batch Status: 7680/60000 (13%) | Loss: 2.305148
Train Epoch: 1 | Batch Status: 8320/60000 (14%) | Loss: 2.292041
Train Epoch: 1 | Batch Status: 8960/60000 (15%) | Loss: 2.294998
Train Epoc